In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import requests
# !pip install openai

import openai
from openai import OpenAI
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /home/lempermi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
#This code initializes a process for parsing and extracting data from an HTML document.
#It starts by defining a list called `sec_list` that contains the names of sections of interest ("Business" and "Risk Factors").
#The code then reads the content of an HTML file named 'apple3.html' located in the 'HTML' directory and uses BeautifulSoup to create a parse tree ('soup') from this content.
#This setup prepares the environment for further processing, such as extracting specific information related to the defined sections from the HTML structure.
sec_list = ["Business", "Risk Factors"]
with open('HTML/apple3.html', 'r') as file:
    html = file.read()
soup = BeautifulSoup(html, 'html.parser')

FileNotFoundError: [Errno 2] No such file or directory: 'HTML/apple3.html'

In [ ]:

#This process effectively extracts and combines the textual content from specified sections of the HTML document,
#delimited by the sections listed in sec_list, for further analysis or processing.
text = ""
for i in range(len(sec_list)-1):
        item_1 = sec_list[i]
        item_2 = sec_list[i+1]
        # soup = BeautifulSoup(html, 'html.parser')
        a_tag = soup.find('a', text=item_1)
        if a_tag:
            href = a_tag['href']
            href = href[1:] #removing hashtag in front of id
        a_tag = soup.find('a', text=item_2)
        if a_tag:
            href_2 = a_tag['href']
            href_2 = href_2[1:] #removing hashtag in front of id
        first_div = soup.find('div', id = href)
        second_div = soup.find('div', id = href_2)
        tag = "div"
        if first_div is None:
            first_div = soup.find('p', id = href)
            second_div = soup.find('p', id = href_2)
            tag = "p"
        if first_div is not None:
            next_div = first_div.find_next_sibling(tag)
            while(next_div != second_div):
                text += next_div.get_text(strip=True)
                next_div = next_div.find_next_sibling(tag)
print(text)

Item
            1.    BusinessCompany
            BackgroundThe
            Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and
            accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week
            period that ends on the last Saturday of September.ProductsiPhoneiPhone®is
            the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone
            14 Pro, iPhone 14, iPhone 13, iPhone SE®,
            iPhone 12 and iPhone 11.MacMac®is
            the Company’s line of personal computers based on its macOS®operating system. The Mac line includes laptops MacBook Air®and
            MacBook Pro®, as
            well as desktops iMac®,
            Mac mini®,
            Mac Studio™ and Mac Pro®.iPadiPad®is
            the Company’s line of multipurpose tablets based on its iPadOS®operating system. The iPad line includes iPad Pro®,
           

/tmp/ipykernel_996495/2564916628.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = soup.find('a', text=item_1)
/tmp/ipykernel_996495/2564916628.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = soup.find('a', text=item_2)


In [ ]:
#The code is tailored to extract and display R&D expense figures from an HTML document structured with Inline XBRL tags,
#specifically focusing on ix:nonfraction elements that match certain attributes indicative of R&D expense data.
#The use of a regular expression to parse the numeric values allows the code to handle various formats of numerical data.
nonfraction_tags = soup.find_all("ix:nonfraction")
count = 0
year_count = 2022
for tag in nonfraction_tags:
    if count == 9:
        break
    attrs = tag.attrs
    if (
        "name" in attrs and
        ((attrs["name"] == "us-gaap:ResearchAndDevelopmentExpense" and
        "unitref" in attrs))
    ):
        values = re.findall(r"\d{1,3}(?:,\d{3})*(?:\.\d+)?", tag.text)
        if values:
            print(values)

['26,251']
['21,914']
['18,752']
['26,251']
['21,914']
['18,752']


In [ ]:
company_name = "apple"  # Replace with the desired company name
prompt = f"""Extract a set of DBpedia topics that match the research and development activities of '{company_name}',
as described in the given text: '{text}'. Provide links to DBpedia pages for each topic. Set up a table with two columns:
the left column should contain the topics in which the company conducts research and development,
and the right column should show an estimated amount of money spent on each topic.

Please estimate the approximate amount of money the company spends on each of these research and development topics,
considering that '{company_name}' spent 39.5 billion dollars on research in 2022.
"""
client = OpenAI(
    # This is the default and can be omitted
    api_key="API_KEY"
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)

{
  "id": "chatcmpl-7YKCcFXL31cI7OBXyDUzzxGoCNLun",
  "object": "chat.completion",
  "created": 1688415166,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Topics | Estimated Amount Spent (in billions of dollars)\n------|------\nSmartphones | 10\nPersonal Computers | 5\nTablets | 4\nWearables | 6\nAccessories | 2\nServices | 4\nIntellectual Property | 3\nOther | 5"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 3053,
    "completion_tokens": 62,
    "total_tokens": 3115
  }
}
Research information for sentence 1:
Topics | Estimated Amount Spent (in billions of dollars)
------|------
Smartphones | 10
Personal Computers | 5
Tablets | 4
Wearables | 6
Accessories | 2
Services | 4
Intellectual Property | 3
Other | 5

